## RED NEURONAL RECURRENTE - RNN

Este es un ejemplo sencillo de una RNN que va a permitir llevar a cabo la predicción de caracteres usando un conjunto de datos (cadena de texto).

Importamos las librerías necesarias

In [1]:
import warnings
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam


2025-01-27 19:52:07.426887: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 19:52:07.438784: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 19:52:07.535108: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 19:52:07.618674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738007527.713396   17851 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738007527.73

Desactivamos los WARNINGS

In [2]:
warnings.filterwarnings("ignore")

### Definición del conjunto de datos y creación del vocabulario

Generamos el conjunto de datos a partir de una frase sencilla y, seguidamente, creamos un vocabulario ordenando el conjunto de carácteres que componen la frase.
Después se crea un diccionario mapeando los caracteres del vocabulario con índices para, a continuación, transformar la secuencia de caracteres que conforman la frase en una secuencia de indices.

In [3]:
# Definimos el conjunto de datos (secuencia de texto)
text = "hello world"
# Crear un vocabulario de caracteres
vocab = sorted(set(text))
vocab_size = len(vocab)

In [4]:
# Crear un diccionario de mapeo de caracteres a índices
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = np.array(vocab)

# Convertir la secuencia de texto a una secuencia de índices
text_as_int = np.array([char_to_idx[c] for c in text])

### PREPARACIÓN DE DATOS

In [5]:
# Preparar los datos de entrenamiento (pares de secuencia de entrada y salida)
seq_length = 4
examples_per_epoch = len(text) - seq_length

In [6]:
# Crear las secuencias de entrada y salida
inputs = np.array([text_as_int[i:i+seq_length] for i in range(examples_per_epoch)])
targets = np.array([text_as_int[i+seq_length] for i in range(examples_per_epoch)])

# Reshape para cumplir con el formato esperado por la RNN
inputs = np.reshape(inputs, (examples_per_epoch, seq_length, 1))

# Usar tf.data.Dataset para manejar los datos
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
dataset = dataset.batch(1, drop_remainder=True)


2025-01-27 19:52:10.854293: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### DISEÑO DE LA ARQUITECTURA DE LA RNN

In [7]:
# Crear el modelo RNN
model = Sequential([
    SimpleRNN(50, input_shape=(seq_length, 1), return_sequences=False),
    Dense(vocab_size, activation='softmax')
])
# Compilar el modelo
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy')

### ENTRENAMIENTO

In [8]:
# Entrenar el modelo
model.fit(dataset, epochs=100)


Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.6673  
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2301
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9884
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8225
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7065
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6207
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5525
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4947
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4427
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3934
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3452
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2970
Epoch 13/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2488
Epoch 14/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2008
Epoch 15/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1535
Epoch 16/100
7/7 ━━━━━━━━━━━━━━━

### PREDICCIÓN

In [9]:
# Función para predecir el siguiente carácter en una secuencia dada
def predict_next_char(model, input_text):
    input_eval = np.array([char_to_idx[c] for c in input_text])
    input_eval = np.reshape(input_eval, (1, len(input_eval), 1))
    prediction = model.predict(input_eval)
    predicted_idx = np.argmax(prediction)
    return idx_to_char[predicted_idx]

# Probar el modelo
input_text = "wor"
predicted_char = predict_next_char(model, input_text)
print(f"Entrada: '{input_text}' -> Siguiente carácter predicho: '{predicted_char}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Entrada: 'wor' -> Siguiente carácter predicho: 'l'
